In [64]:
import pandas as pd
import nltk
import re
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
from nltk.stem.porter import *

cleaned_df = pd.read_csv("cleaned_phone_data.csv") 
cleaned_df.head()

,date,lang,country,source,domain,score,score_max,extract,author,product,phone_model
0,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,"As a diehard Samsung fan who has had every Samsung phone since before the S series started, this has been my favorite upgrade so far. I am amazed at some of the reviews and think people may just have a defective device that needs to be replaced. The battery life is amazing.",CarolAnn35,Samsung Galaxy S8,Samsung Galaxy S8
1,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth and beautiful I highly recommend the phone you will not regret getting this phone.,james0923,Samsung Galaxy S8,Samsung Galaxy S8
2,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sluggish for the few apps that I have installed. Samsung wants to vacuum all of your data with their apps that requires more access than needed. What else is new.,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unlocked (GSM + CDMA, US WARRANTY) (Orchid Gray)",Samsung Galaxy S8
3,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,"Never disappointed. One of the reasons I've been a long Samsung (product) consumer, they're ability to impress me with each product release. I've owned, or maintained, all Samsung smartphones starting with the S4 and the S8 is the best one yet.",Buster2020,Samsung Galaxy S8 64GB (AT&T),Samsung Galaxy S8
4,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people that have carried their phone in their pocket without problems until the S8. Day one screen has crack from being in my pocket. Bluetooth on my 1st trip struggles to stay connected. Good news.....battery life is great and lots of great features.,S Ate Mine,Samsung Galaxy S8,Samsung Galaxy S8


In [65]:
import nltk
import re
import gensim

def clean_extract(extract_data):
    stop_list = nltk.corpus.stopwords.words('english')
    tokenizer = nltk.tokenize.word_tokenize
    stemmer = nltk.stem.porter.PorterStemmer()
    
    extract_list  = [tokenizer(extract) for extract in extract_data]
    extract_list = [[w.lower() for w in extract] for extract in extract_list]
    extract_list = [[w for w in extract if re.search('^[a-z]+$',w)] for extract in extract_list]
    extract_list = [[w for w in extract if w not in stop_list] for extract in extract_list]
    extract_list = [[stemmer.stem(w) for w in extract] for extract in extract_list]
    
    return extract_list

extract_list = cleaned_df['extract'].values.tolist()
extract_words = clean_extract(extract_list)
extract_sent = [' '.join(extract) for extract in extract_words]


print(extract_sent[5])

type person never would complet sold phone realli samsung galaxi nexu googl phone got time fri made sens


In [66]:
print(extract_list[1:4])
print(extract_sent[1:4])

['Love the phone. the phone is sleek and smooth and beautiful I highly recommend the phone you will not regret getting this phone.', "Adequate feel. Nice heft. Processor's still sluggish for the few apps that I have installed. Samsung wants to vacuum all of your data with their apps that requires more access than needed. What else is new.", "Never disappointed. One of the reasons I've been a long Samsung (product) consumer, they're ability to impress me with each product release. I've owned, or maintained, all Samsung smartphones starting with the S4 and the S8 is the best one yet."]
['love phone phone sleek smooth beauti highli recommend phone regret get phone', 'adequ feel nice heft processor still sluggish app instal samsung want vacuum data app requir access need els new', 'never disappoint one reason long samsung product consum abil impress product releas own maintain samsung smartphon start best one yet']


In [67]:
poswords_df = pd.read_csv('huliu_positive-words.txt', sep=" ", header=None, encoding='latin-1')
#poswords_df.head()
poswords_list = poswords_df[0].values.tolist()

negwords_df = pd.read_csv('huliu_negative-words.txt', sep=" ", header=None, encoding='latin-1')
#negwords_df.head()
negwords_list = negwords_df[0].values.tolist()

print(poswords_list[0:10])
print(negwords_list[0:10])

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


## TRAIL ONE (WITH YL'S CLEANING)

In [68]:
#nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk import word_tokenize

def simple_sentiment(text):
    pos_words = 0
    neg_words = 0
    tokens = [word.lower() for word in word_tokenize(text)]

    for word in tokens:
        if word in opinion_lexicon.positive():
            pos_words += 1
        elif word in opinion_lexicon.negative():
            neg_words += 1
        
    return pos_words-neg_words

#    if pos_words > neg_words:
#        return 'Positive'
#    elif pos_words < neg_words:
#        return 'Negative'
#    elif pos_words == neg_words:
#        return 'Neutral'
    
#for sentence in extract_sent[1:4]:
#    print(simple_sentiment(sentence))

sentence_positivity_counts=[]
for sentence in extract_sent[0:4]:
    sentence_positivity_counts.append(simple_sentiment(sentence))

sentence_positivity_counts

[-1, 3, 0, 1]

In [69]:
result = pd.DataFrame(
    {'sentence': extract_sent[0:4],
     'sentiment': sentence_positivity_counts
    })

pd.set_option('max_colwidth', 800)

print(result[0:10])

                                                                                                                                            sentence  \
0  diehard samsung fan everi samsung phone sinc seri start favorit upgrad far amaz review think peopl may defect devic need replac batteri life amaz   
1                                                                       love phone phone sleek smooth beauti highli recommend phone regret get phone   
2                                   adequ feel nice heft processor still sluggish app instal samsung want vacuum data app requir access need els new   
3              never disappoint one reason long samsung product consum abil impress product releas own maintain samsung smartphon start best one yet   

   sentiment  
0         -1  
1          3  
2          0  
3          1  


## TRAIL TWO (WITHOUT YL'S CLEANING)

In [70]:
from nltk.corpus import opinion_lexicon
from nltk import word_tokenize

def simple_sentiment(text):
    pos_words = 0
    neg_words = 0
    tokens = [word.lower() for word in word_tokenize(text)]

    for word in tokens:
        if word in opinion_lexicon.positive():
            pos_words += 1
        elif word in opinion_lexicon.negative():
            neg_words += 1
        
    return pos_words-neg_words

sentence_positivity_counts_2=[]
for sentence in extract_list[0:4]:
    sentence_positivity_counts_2.append(simple_sentiment(sentence))

sentence_positivity_counts_2

[2, 4, 1, 1]

In [71]:
result2 = pd.DataFrame(
    {'sentence': extract_list[0:4],
     'sentiment': sentence_positivity_counts_2
    })


print(result2[0:10])

                                                                                                                                                                                                                                                                             sentence  \
0  As a diehard Samsung fan who has had every Samsung phone since before the S series started, this has been my favorite upgrade so far. I am amazed at some of the reviews and think people may just have a defective device that needs to be replaced. The battery life is amazing.   
1                                                                                                                                                    Love the phone. the phone is sleek and smooth and beautiful I highly recommend the phone you will not regret getting this phone.   
2                                                                      Adequate feel. Nice heft. Processor's still sluggish for the few apps that I have inst